In [4]:
pip install shap

Note: you may need to restart the kernel to use updated packages.


In [5]:
#importing packages
import shap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import requests                                      # reading data
from io import StringIO

from sklearn.datasets import fetch_openml            # common data set access
from sklearn.preprocessing import StandardScaler     # scaling transform
from sklearn.model_selection import train_test_split # validation tools
from sklearn.metrics import zero_one_loss as J01
from sklearn.preprocessing import OneHotEncoder
import xgboost
from sklearn.linear_model import LinearRegression

import sklearn.tree as tree

# Fix the random seed for reproducibility
# !! Important !! : do not change this
seed = 1234
np.random.seed(seed)  

ModuleNotFoundError: No module named 'xgboost'

In [18]:
# Reading in the CSV file
Xtr = pd.read_csv("data/X_TRAINING_SET_Diabetes_ENCODED.csv")
Ytr = pd.read_csv("data/Y_TRAINING_SET.csv")
                  
Xtest = pd.read_csv("data/X_VALIDATION_SET_ENCODED.csv")
Ytest = pd.read_csv("data/Y_VALIDATION_SET.csv")




In [19]:
Ytr

,Unnamed: 0,readmitted
0,47866,NO
1,24673,NO
2,61033,NO
3,74444,>30
4,56971,<30
...,...,...
71231,89460,NO
71232,60620,NO
71233,34086,NO
71234,58067,>30


In [20]:
Ytr['readmitted'] = Ytr['readmitted'].map({'<30': 1, 'NO': 0, '>30': 2})
Ytr.to_csv('data/Y_TRAINING_SET_EDIT.csv', index=False)

In [21]:
Ytest

,Unnamed: 0,readmitted
0,60056,NO
1,66252,NO
2,71804,<30
3,16676,NO
4,25765,NO
...,...,...
10069,91655,NO
10070,28636,>30
10071,101193,NO
10072,94482,NO


In [22]:
Ytest['readmitted'] = Ytest ['readmitted'].map({'<30': 1, 'NO': 0, '>30': 2})
Ytest.to_csv('data/Y_VALIDATION_SET_EDIT.csv', index=False)

In [23]:
Ytest

,Unnamed: 0,readmitted
0,60056,0
1,66252,0
2,71804,1
3,16676,0
4,25765,0
...,...,...
10069,91655,0
10070,28636,2
10071,101193,0
10072,94482,0
